In [1]:
import numpy as np
import pandas as pd
import re
import glob
from   os import path

from   transformers import AutoTokenizer, AutoModelForTokenClassification
from   transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model     = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp       = pipeline("ner", model=model, tokenizer=tokenizer)

In [2]:
def combineHeadlineText(row):
    if isinstance(row["Headline"], str):
        return row["Headline"] + ". " + row["Text"]
    else:
        return row["Text"]

In [8]:
lst_companies = [] 
lst_files = [] 

Path = "*.csv"
count = 0

for fname in glob.glob(Path):
    lst_files.append(fname)

lst_files = sorted(lst_files)

for file in lst_files:
    if not path.isfile('/Users/mayagolan/OneDrive - Imperial College London/EE4/FYP/Final-Year-Project-main/Pipeline/TMF NER/{0}'.format(file)):
        print(file)
        df = pd.read_csv(file)
        if df.shape[0] != 0:
            df = df.drop(['Unnamed: 0', 'Author'], axis = 1)
            df = df.drop_duplicates(['Date','Headline'],keep= 'last')
            df['Text'] = df['Text'].astype(str)  
            df['Text'] = df.apply(lambda row: combineHeadlineText(row), axis=1)
            df['Date'] = df['Date'].str.replace(r'Published: ', ' ')
            df['Date'] = df['Date'].str.replace(r'Updated: ', ' ')
            df['Date'] = pd.to_datetime(df['Date'].str[-23:], utc=True).dt.date           #change date format to YYYY-MM-DD
            df = df.sort_values(by = ['Date'], ascending = True)

            count = []
            company_name = file[0:len(file)-4].strip()

            for i in range(len(df)):
                ner_results = nlp(df["Text"].iloc[i])
                company_instances = [d for d in ner_results if (d['entity'] in "B-ORG") and (d['word'] in company_name) and (d['score'] > 0.98)]
                count.append(len(company_instances))

            df['Count'] = count
            df = df[df['Count'] > 0]
            df = df.drop(['Count'], axis = 1)
#             print(company_name)
            df.to_csv(r'/Users/mayagolan/OneDrive - Imperial College London/EE4/FYP/Final-Year-Project-main/Pipeline/TMF NER/{0}.csv'.format(company_name))

Antofagasta Plc.csv
Ashland Inc..csv
BED Bath & Beyond.csv
Deutsche Telekom AG.csv
Fortis Inc.csv
Retail Properties of America Inc..csv
Seagate Technology .csv
Solvay Finance America LLC.csv
Weibo Corporation.csv
Xcel Energy Inc.csv
